In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from factor_analyzer.factor_analyzer import calculate_kmo

In [2]:
# origin=read_excel('행정동병합테스트22.xlsx')
origin=read_excel('행정동병합테스트22(개발제한행정동제외).xlsx')
origin.head()

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355


In [3]:
origin.isna().sum()
origin1=origin.fillna(0)
scaler=MinMaxScaler()
origin1.columns

Index(['Unnamed: 0', '시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명',
       '생성일자', '말소일자', '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)',
       '구성비 (%)', '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수',
       '개소 (공영+민영)', '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)',
       '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수',
       '행정동별유통업체수', '공공자전거대여소수', '인구밀도'],
      dtype='object')

In [4]:
df = origin1.copy()
# df['면적minmax']=scaler.fit_transform(df[['면적 (k㎡)']])
df['인구밀도minmax']=scaler.fit_transform(df[['인구밀도']])
df['공공체육시설minmax']=scaler.fit_transform(df[['공공체육시설_수']])
df['문화시설minmax']=scaler.fit_transform(df[['문화시설_수']])
df['유통업체minmax']=scaler.fit_transform(df[['행정동별유통업체수']])
df.head()

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,인구밀도minmax,공공체육시설minmax,문화시설minmax,유통업체minmax
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,0.352898,0,32,0,3,4552.529183,0.081563,0.256410,0.055556,0.0
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,0.063554,0,27,0,12,7611.382114,0.137296,0.205128,0.120370,0.0
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,1.000000,0,16,0,5,1728.859060,0.030114,0.051282,0.287037,0.0
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,0.000000,10,34,0,5,4130.837004,0.073879,0.128205,0.046296,0.0
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,0.001192,10,54,0,5,1990.755355,0.034886,0.128205,0.101852,0.0


In [5]:
df['여가생활지수'] = (df['공공체육시설minmax'] + df['문화시설minmax'] + df['유통업체minmax']) / df['인구밀도minmax']
df.head()

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,인구밀도minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,0,32,0,3,4552.529183,0.081563,0.256410,0.055556,0.0,3.824858
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,0,27,0,12,7611.382114,0.137296,0.205128,0.120370,0.0,2.370774
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,0,16,0,5,1728.859060,0.030114,0.051282,0.287037,0.0,11.234528
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,10,34,0,5,4130.837004,0.073879,0.128205,0.046296,0.0,2.361980
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,10,54,0,5,1990.755355,0.034886,0.128205,0.101852,0.0,6.594519


In [18]:
df['여가생활지수'].value_counts().sort_index()

여가생활지수
0.000000     37
0.012994      1
0.013266      1
0.015051      1
0.015141      1
             ..
6.594519      1
6.760783      1
11.234528     1
12.399563     1
61.487991     1
Name: count, Length: 369, dtype: int64

In [24]:
df['여가생활지수'].max()

61.48799114829103

In [7]:
df['여가생활지수'].isna().sum()

1

In [8]:
df.columns

Index(['Unnamed: 0', '시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명',
       '생성일자', '말소일자', '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)',
       '구성비 (%)', '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수',
       '개소 (공영+민영)', '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)',
       '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수',
       '행정동별유통업체수', '공공자전거대여소수', '인구밀도', '인구밀도minmax', '공공체육시설minmax',
       '문화시설minmax', '유통업체minmax', '여가생활지수'],
      dtype='object')

In [9]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','공공체육시설수','문화시설수','행정동별유통업체수','인구밀도','인구밀도minmax','공공체육시설minmax','문화시설minmax','유통업체minmax','여가생활지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,행정동별유통업체수,인구밀도,인구밀도minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,0,4552.529183,0.081563,0.256410,0.055556,0.0,3.824858
1,11110,종로구,1111053000,사직동,0,7611.382114,0.137296,0.205128,0.120370,0.0,2.370774
2,11110,종로구,1111054000,삼청동,0,1728.859060,0.030114,0.051282,0.287037,0.0,11.234528
3,11110,종로구,1111055000,부암동,0,4130.837004,0.073879,0.128205,0.046296,0.0,2.361980
4,11110,종로구,1111056000,평창동,0,1990.755355,0.034886,0.128205,0.101852,0.0,6.594519


In [10]:
df1.isna().sum()

시군구코드           0
시군구명            0
행정동코드           0
읍면동명            0
행정동별유통업체수       0
인구밀도            0
인구밀도minmax      0
공공체육시설minmax    0
문화시설minmax      0
유통업체minmax      0
여가생활지수          1
dtype: int64

In [11]:
df2=df1.fillna(0)
df2.isna().sum()

시군구코드           0
시군구명            0
행정동코드           0
읍면동명            0
행정동별유통업체수       0
인구밀도            0
인구밀도minmax      0
공공체육시설minmax    0
문화시설minmax      0
유통업체minmax      0
여가생활지수          0
dtype: int64

In [12]:
df2['여가생활지수'].value_counts().sort_index()

여가생활지수
0.000000     38
0.012994      1
0.013266      1
0.015051      1
0.015141      1
             ..
6.594519      1
6.760783      1
11.234528     1
12.399563     1
61.487991     1
Name: count, Length: 369, dtype: int64

In [13]:
# df2['여가생활지수'].replace([np.inf, -np.inf], 0, inplace=True)
# df2

In [14]:
df2['여가생활지수minmax']=scaler.fit_transform(df2[['여가생활지수']])

In [17]:
df2

,시군구코드,시군구명,행정동코드,읍면동명,행정동별유통업체수,인구밀도,인구밀도minmax,공공체육시설minmax,문화시설minmax,유통업체minmax,여가생활지수,여가생활지수minmax
0,11110,종로구,1111051500,청운효자동,0,4552.529183,0.081563,0.256410,0.055556,0.0000,3.824858,0.062205
1,11110,종로구,1111053000,사직동,0,7611.382114,0.137296,0.205128,0.120370,0.0000,2.370774,0.038557
2,11110,종로구,1111054000,삼청동,0,1728.859060,0.030114,0.051282,0.287037,0.0000,11.234528,0.182711
3,11110,종로구,1111055000,부암동,0,4130.837004,0.073879,0.128205,0.046296,0.0000,2.361980,0.038414
4,11110,종로구,1111056000,평창동,0,1990.755355,0.034886,0.128205,0.101852,0.0000,6.594519,0.107249
...,...,...,...,...,...,...,...,...,...,...,...,...
401,11740,강동구,1174065000,성내제2동,1,34780.597015,0.632331,0.000000,0.018519,0.0625,0.128127,0.002084
402,11740,강동구,1174066000,성내제3동,1,32228.169014,0.585825,0.000000,0.009259,0.0625,0.122493,0.001992
403,11740,강동구,1174068500,길동,2,28038.509317,0.509487,0.025641,0.009259,0.1250,0.313845,0.005104
404,11740,강동구,1174069000,둔촌제1동,0,76.086957,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000


In [16]:
df2.to_excel('여가생활지수1.xlsx')